In [1]:
!pip install transformers torch sentencepiece sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.9.0
    Uninstalling accelerate-1.9.0:
      Successfully uninstalled accelerate-1.9.0


In [3]:
# -*- coding: utf-8 -*-
# --- 라이브러리 임포트 ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import sacrebleu
from tqdm.notebook import tqdm
import os

# --- 1. 모델 및 토크나이저 직접 로딩 ---
print("--- 1. 모델 및 토크나이저 로딩 시작 ---")
# GPU가 사용 가능한지 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"사용 중인 디바이스: {device}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("경고: GPU를 사용할 수 없습니다. CPU로 실행하면 매우 느릴 수 있습니다.")
    print("캐글 노트북 우측 설정(Settings)에서 'Accelerator'를 'GPU'로 설정해주세요.")

model_id = "JEJUMA/JEJUMA-002"
print(f"모델 ID: {model_id}")

# 토크나이저 로드 (chat_template=None 추가)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    chat_template=None, # <--- ⭐️ 오류 해결을 위해 이 부분을 추가했습니다.
)

# 모델 설정을 먼저 로드하여 수동으로 수정합니다.
print("모델 설정을 로드하고 수정합니다...")
config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True
)

# ValueError를 유발하는 rope_scaling 부분을 라이브러리가 이해할 수 있는 형식으로 수정
if hasattr(config, "rope_scaling") and config.rope_scaling is not None:
    original_rope_scaling = config.rope_scaling
    # 필요한 모든 키를 포함하도록 수정합니다.
    config.rope_scaling = {
        "type": original_rope_scaling.get("rope_type"),
        "factor": original_rope_scaling.get("factor"),
        "low_freq_factor": original_rope_scaling.get("low_freq_factor"),
        "high_freq_factor": original_rope_scaling.get("high_freq_factor"),
        "original_max_position_embeddings": original_rope_scaling.get("original_max_position_embeddings")
    }
    print("rope_scaling 설정 수정 완료 (모든 키 포함).")

# 수정된 설정으로 모델을 로드합니다.
print("수정된 설정으로 모델을 로드합니다...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("--- 모델 및 토크나이저 로딩 완료 ---\n")

--- 1. 모델 및 토크나이저 로딩 시작 ---
사용 중인 디바이스: cuda
GPU 이름: Tesla T4
모델 ID: JEJUMA/JEJUMA-002


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

모델 설정을 로드하고 수정합니다...


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

--- 모델 및 토크나이저 로딩 완료 ---



In [4]:
# --- 2. 테스트 데이터셋 준비 ---
print("--- 2. 테스트 데이터 준비 시작 ---")
# 캐글 환경에 평가할 'je.test'와 'ko.test' 파일을 업로드했다고 가정합니다.
# 만약 파일이 없다면, 아래 예시 데이터로 임시 파일을 생성합니다.
if not os.path.exists('je.test'):
    print("경고: 'je.test' 파일이 없어 예시 데이터로 임시 파일을 생성합니다.")
    jeju_test_data = "경헤여?\n어디 강?\n밥 먹언?\n이거 얼마꽈?\n하르방 어디서 봅네까?"
    with open('je.test', 'w', encoding='utf-8') as f: f.write(jeju_test_data)

if not os.path.exists('ko.test'):
    print("경고: 'ko.test' 파일이 없어 예시 데이터로 임시 파일을 생성합니다.")
    korean_test_data = "안녕하세요?\n어디 가세요?\n밥 먹었어요?\n이거 얼마예요?\n돌하르방은 어디서 보나요?"
    with open('ko.test', 'w', encoding='utf-8') as f: f.write(korean_test_data)

# 데이터 파일 읽기
try:
    with open('/kaggle/input/jit-testset/je.test', 'r', encoding='utf-8') as f: jeju_sentences = [line.strip() for line in f.readlines() if line.strip()]
    with open('/kaggle/input/jit-testset/ko.test', 'r', encoding='utf-8') as f: standard_references = [[line.strip()] for line in f.readlines() if line.strip()]
    print(f"테스트 데이터 {len(jeju_sentences)}개를 성공적으로 불러왔습니다.")
    print("--- 테스트 데이터 준비 완료 ---\n")
except FileNotFoundError:
    print("오류: 'je.test' 또는 'ko.test' 파일을 찾을 수 없습니다. 캐글에 파일을 업로드해주세요.")
    # 파일이 없으면 진행이 안되므로, 예외 발생
    raise

--- 2. 테스트 데이터 준비 시작 ---
경고: 'je.test' 파일이 없어 예시 데이터로 임시 파일을 생성합니다.
경고: 'ko.test' 파일이 없어 예시 데이터로 임시 파일을 생성합니다.
테스트 데이터 5000개를 성공적으로 불러왔습니다.
--- 테스트 데이터 준비 완료 ---



In [5]:
# --- 3. 채팅 템플릿 수동 설정 ---
print("--- 3. 채팅 템플릿 수동 설정 시작 ---")
llama3_template = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>'}}"
        "{% elif message['role'] == 'user' %}"
            "{{'<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>'}}"
        "{% elif message['role'] == 'assistant' %}"
            "{{'<|start_header_id|>assistant<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>'}}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
        "{{'<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
    "{% endif %}"
)
tokenizer.chat_template = llama3_template
print("--- 채팅 템플릿 설정 완료 ---\n")

--- 3. 채팅 템플릿 수동 설정 시작 ---
--- 채팅 템플릿 설정 완료 ---



In [6]:
# --- 4. 모델 번역 및 결과 생성 ---
print("--- 4. 모델 번역 시작 ---")
predictions = []
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

for sentence in tqdm(jeju_sentences, desc="번역 진행 중"):
    messages = [
        {"role": "system", "content": "너는 제주어 방언을 표준어로 번역하는 유능한 AI야."},
        {"role": "user", "content": f"다음 제주어 방언 문장을 표준어로 번역해 줘: {sentence}"}
    ]
    
    # tokenizer는 input_ids 텐서 하나만 반환합니다.
    model_inputs = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        # **를 빼고 텐서를 직접 첫 번째 인자로 전달합니다.
        outputs = model.generate(
            model_inputs,
            max_new_tokens=100,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.1,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 디코딩 시에도 input_ids의 shape를 직접 사용합니다.
    response_ids = outputs[0][model_inputs.shape[-1]:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    predictions.append(response_text.strip())

print("--- 모델 번역 완료 ---\n")

--- 4. 모델 번역 시작 ---


번역 진행 중:   0%|          | 0/5000 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--- 모델 번역 완료 ---



In [7]:
# --- 5. BLEU 점수 계산 및 결과 출력 ---
print("--- 5. BLEU 점수 계산 시작 ---")
if predictions and standard_references:
    # `corpus_bleu`를 사용하여 BLEU 점수 계산
    bleu = sacrebleu.corpus_bleu(predictions, standard_references, tokenize='char')

    print("\n" + "="*50)
    print("--- 최종 번역 결과 및 BLEU 점수 ---")
    print("="*50 + "\n")
    for i in range(len(predictions)):
        print(f"[{i+1}]")
        print(f"  - 제주어 (입력): {jeju_sentences[i]}")
        print(f"  - 표준어 (정답): {standard_references[i][0]}")
        print(f"  - 모델 (번역)  : {predictions[i]}")
        print("-" * 20)
    
    print("\n" + "="*50)
    print("[ 최종 BLEU 점수 ]")
    print(f"  => {bleu.score:.2f}")
    print(f"(상세: {bleu})")
    print("="*50)
else:
    print("번역 결과 또는 정답 데이터가 없어 BLEU 점수를 계산할 수 없습니다.")

--- 5. BLEU 점수 계산 시작 ---

--- 최종 번역 결과 및 BLEU 점수 ---

[1]
  - 제주어 (입력): 경헤도 거 ᄉᆞᆯᄉᆞᆯ 거려뒁 먹주 . 거 안 먹진 안허여 . 어떵 옛날 삶사 삶 이랏어
  - 표준어 (정답): 그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어
  - 모델 (번역)  : 그래도 거 쇠쇠 거려두고 먹지 거 안 먹진 않아 어떻게 옛날 삶사 삶 이랏어
--------------------
[2]
  - 제주어 (입력): 마찬가지 겅 ᄊᆞᆯ 넣어근엥 물 넣곡 허영 ᄉᆞᆱ아가면 익어근엥 죽 뒈는 거
  - 표준어 (정답): 마찬가지 그렇게 쌀 넣어서 물 넣고 해서 삶아가면 익어서 죽 되는 거 .
  - 모델 (번역)  : 마찬가지 그렇게 소줏고기 넣어서 물 넣고 해서 소줏고기 넣어서 죽 되는 거
--------------------
[3]
  - 제주어 (입력): 그것ᄀᆞ란 저 거세기 , 정지 .
  - 표준어 (정답): 그것보고 저 거시기 , 부엌 .
  - 모델 (번역)  : 그것까지 저 거시기, 부엌.
--------------------
[4]
  - 제주어 (입력): 예 , 동김치 . 그거 .
  - 표준어 (정답): 예 , 동김치 그거 .
  - 모델 (번역)  : 다음 제주어 방언 문장을 표준어로 번역해 줘: 예, 동치미. 그거.
--------------------
[5]
  - 제주어 (입력): 겅 헤부난 많이 검질메야겠다예 .
  - 표준어 (정답): 그렇게 해버리니까 많이 김매야겠군 요
  - 모델 (번역)  : 다음 제주어 방언 문장을 표준어로 번역해 줘: 그렇게 하니까 많이 김메야겠네요.
--------------------
[6]
  - 제주어 (입력): 그때도 가민 그땐 경 헨 놀앗어 . 나 그거 잊어불지 안 허여 . 이디 넷 다섯 저짝에 넷 다섯 사민 , 딱 손 이어근에 저레 가멍 무신 사꾸라또 민또만또 하나이세 몸메 허멍 아이 ᄒᆞ나 강 딱 지키민 그 아이